In [ ]:
%load_ext autoreload
%autoreload 2

In [10]:
from simulator.game.connect import Config, State

from alphazero_implementation.models.games.connect4 import CNNModel

config = Config(6, 7, 4)


path = "/Users/pveron/Code/alphazero-implementation/checkpoints/run_156/model-epoch=1999.ckpt"
model = CNNModel.load_from_checkpoint(  # type: ignore[arg-type]
    path,
    height=config.height,
    width=config.width,
    max_actions=config.width,
    num_players=config.num_players,
).eval()


In [11]:
state = config.sample_initial_state()
json = {
    "config": {"count": 4, "height": 6, "width": 7},
    "grid": [
        [0, 0, 0, -1, -1, 1, -1],
        [-1, -1, -1, -1, -1, 1, -1],
        [-1, -1, -1, -1, -1, 1, -1],
        [-1, -1, -1, -1, -1, -1, -1],
        [-1, -1, -1, -1, -1, -1, -1],
        [-1, -1, -1, -1, -1, -1, -1],
    ],
    "player": 0,
}

state = State.from_json(json)

In [ ]:
from alphazero_implementation.core.training import EpisodeGenerator

mcts = EpisodeGenerator(
    model=model,
    num_simulations=100,
    num_episodes=1,
    game_initial_state=state,
)

[episode] = mcts.generate_episodes()

for sample in episode.samples:
    print(sample.state.grid)
    for action, prob in sample.policy.items():
        print(f"{action.column}: {prob}")
    print(sample.value)


In [13]:
import json

with open("episode.json", "w") as f:
    json.dump(episode.to_dict(), f)

In [14]:
from alphazero_implementation.core.training.episode import Episode

episode = Episode.from_dict(json.load(open("episode.json")))

In [ ]:
episode